<a href="https://colab.research.google.com/github/madhusudanhasbe/Natural_Language_Processing_Lab/blob/main/22070126061_NLP_Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 5

Name: Madhusudan Hasbe\
PRN: 22070126061\
TY AIML A3

Github Link: https://github.com/madhusudanhasbe/Natural_Language_Processing_Lab/blob/main/22070126061_NLP_Assignment_5.ipynb

## Question Answering by Fine-Tuning T5

#### Imports

In [3]:
!pip install datasets

In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW, get_scheduler
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import os
from google.colab import drive
from nltk.translate.bleu_score import sentence_bleu
import nltk

# Download NLTK data for BLEU score calculation
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
# Define the directory to save the model in Google Drive
save_directory = '/content/drive/MyDrive/T5_finetuned_CoQA'
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

#### Load Dataset and Process Dataset

In [7]:
# Load the CoQA dataset
coqa = load_dataset('coqa')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.17k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/793k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7199 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [8]:
# Custom Dataset Class for CoQA
class CoQADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question = item['questions'][0]
        context = item['story']
        answer = item['answers']['input_text'][0]  # Take the first answer (CoQA provides multiple answers)

        # Input: question + context
        input_text = f"question: {question}  context: {context} </s>"
        inputs = self.tokenizer(
            input_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt"
        )

        # Target: Answer
        targets = self.tokenizer(
            answer, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt"
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': targets['input_ids'].squeeze(),
        }

In [9]:
# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
# Prepare data for training
train_data = coqa['train']
train_dataset = CoQADataset(train_data, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [11]:
# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_loader) * 3  # 3 epochs
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# Initialize variables for tracking the best model
best_loss = float('inf')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#### Training Function

In [12]:
# Training loop with tqdm progress bar and model saving
model.train()
for epoch in range(3):  # Adjust number of epochs as needed
    epoch_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}", leave=True)

    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        epoch_loss += loss.item()
        progress_bar.set_postfix({"Loss": loss.item()})

    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch + 1} finished. Average Loss: {avg_loss}")

    # Save the model if it's the best so far
    if avg_loss < best_loss:
        best_loss = avg_loss
        model.save_pretrained(save_directory)
        tokenizer.save_pretrained(save_directory)
        print(f"Best model saved with avg_loss: {avg_loss}")

print(f"Training complete. Best model saved at: {save_directory}")

Epoch 1:   0%|          | 0/900 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
Epoch 1: 100%|██████████| 900/900 [10:06<00:00,  1.48it/s, Loss=0.046]


Epoch 1 finished. Average Loss: 0.5551380747887823
Best model saved with avg_loss: 0.5551380747887823


Epoch 2:   0%|          | 0/900 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
Epoch 2: 100%|██████████| 900/900 [10:08<00:00,  1.48it/s, Loss=0.0265]


Epoch 2 finished. Average Loss: 0.03891403283716904
Best model saved with avg_loss: 0.03891403283716904


Epoch 3:   0%|          | 0/900 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
Epoch 3: 100%|██████████| 900/900 [10:09<00:00,  1.48it/s, Loss=0.023]


Epoch 3 finished. Average Loss: 0.023379831961873505
Best model saved with avg_loss: 0.023379831961873505
Training complete. Best model saved at: /content/drive/MyDrive/T5_finetuned_CoQA


#### BLEU Score

In [13]:
# Function to generate an answer from the model for a single input
def generate_answer(model, tokenizer, input_text, device, max_length=512):
    # Tokenize input text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=max_length, truncation=True).to(device)

    # Generate answer
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], max_length=max_length, num_beams=2, early_stopping=True)

    # Decode the answer from token IDs to text
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Function to calculate BLEU score for a single prediction and reference
def calculate_bleu(prediction, reference):
    if isinstance(reference, list):
      reference = reference[0]
    return sentence_bleu([reference.split()], prediction.split())

# Example usage: Test on a single question + context example
def test_single_example(model, tokenizer, device, question, context, reference_answer):
    if isinstance(question, list):
        question = question[0]
    if isinstance(context, list):
        context = context[0]
    if isinstance(reference_answer, list):
        reference_answer = reference_answer[0]
    # Prepare input: question + context
    input_text = f"question: {question}  context: {context}"

    # Generate the prediction
    predicted_answer = generate_answer(model, tokenizer, input_text, device)

    # Calculate BLEU score
    bleu_score = calculate_bleu(predicted_answer, reference_answer)

    # Print prediction and BLEU score
    print(f"Predicted Answer: {predicted_answer}")
    print(f"Reference Answer: {reference_answer}")
    print(f"BLEU Score: {bleu_score:.4f}")

    return bleu_score

In [14]:
question = coqa['train']['questions'][7]
context = coqa['train']['story']
reference_answer = coqa['train']['answers'][0]['input_text'][7]

bleu_score = test_single_example(model, tokenizer, device, question, context, reference_answer)

Predicted Answer: .
Reference Answer: The Vatican Apostolic Library
BLEU Score: 0.0000


## Question Answering by Fine-Tuning DistilBERT

#### Imports

In [50]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
import nltk

# Download NLTK data for BLEU score calculation
nltk.download('punkt')

# Ignore Warnings
import logging
logging.disable(logging.WARNING)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Load Dataset and Process Dataset

In [51]:
# Load CoQA dataset
def load_coqa_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data['data']

# Custom dataset class
class CoQADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        context = item['story']
        question = item['questions'][0]['input_text']
        answer = item['answers'][0]['input_text']

        # Tokenize the input
        inputs = self.tokenizer.encode_plus(
            question,
            context,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Find the start and end positions of the answer in the tokenized input
        input_ids = inputs['input_ids'][0]
        answer_tokens = self.tokenizer.encode(answer, add_special_tokens=False)
        start_position = None
        end_position = None

        for i in range(len(input_ids) - len(answer_tokens) + 1):
            if input_ids[i:i+len(answer_tokens)].tolist() == answer_tokens:
                start_position = i
                end_position = i + len(answer_tokens) - 1
                break

        # If the answer is not found, use the CLS token position as a default
        if start_position is None:
            start_position = 0
            end_position = 0

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'start_positions': torch.tensor(start_position),
            'end_positions': torch.tensor(end_position),
            'answer': answer
        }

In [52]:
data = load_coqa_data('/content/drive/MyDrive/coqa-train-v1.0.json')
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

In [53]:
# Initialize tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Prepare datasets and dataloaders
train_dataset = CoQADataset(train_data, tokenizer)
val_dataset = CoQADataset(val_data, tokenizer)
test_dataset = CoQADataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

#### Training Function

In [54]:
# Training function
def train(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc="Training")
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix({'loss': loss.item()})
    return total_loss / len(train_loader)

In [55]:
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

# Set device and move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#### Validation and Testing Function

In [56]:
# Validation function
def validate(model, val_loader, device):
    model.eval()
    total_loss = 0
    progress_bar = tqdm(val_loader, desc="Validating")
    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
            loss = outputs.loss
            total_loss += loss.item()
            progress_bar.set_postfix({'loss': loss.item()})
    return total_loss / len(val_loader)

In [57]:
# Test function
def test(model, test_loader, tokenizer, device):
    model.eval()
    all_predictions = []
    all_answers = []
    progress_bar = tqdm(test_loader, desc="Testing")
    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            answers = batch['answer']
            outputs = model(input_ids, attention_mask=attention_mask)
            start_scores = outputs.start_logits
            end_scores = outputs.end_logits
            for i in range(input_ids.shape[0]):
                start_index = torch.argmax(start_scores[i])
                end_index = torch.argmax(end_scores[i])
                prediction = tokenizer.decode(input_ids[i][start_index:end_index+1])
                all_predictions.append(prediction)
                all_answers.append(answers[i])
    bleu_score = calculate_bleu(all_predictions, all_answers)
    return bleu_score

#### Training

In [58]:
import os

num_epochs = 3
best_loss = float('inf')
save_path = '/content/drive/My Drive/distilbert_qa_model.pth'

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss = train(model, train_loader, optimizer, device)
    val_loss = validate(model, val_loader, device)
    print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), save_path)
        print("Model saved to Google Drive!")
    else:
        print("Validation Loss Increased. Model Not Saved.")
    print("*" * 50)

Epoch 1/3


Validating: 100%|██████████| 135/135 [00:24<00:00,  5.41it/s, loss=2.2]


Train Loss: 2.7784, Validation Loss: 2.1292
Model saved to Google Drive!
**************************************************
Epoch 2/3


Validating: 100%|██████████| 135/135 [00:24<00:00,  5.42it/s, loss=1.97]


Train Loss: 1.4381, Validation Loss: 1.9779
Model saved to Google Drive!
**************************************************
Epoch 3/3


Validating: 100%|██████████| 135/135 [00:24<00:00,  5.40it/s, loss=1.96]

Train Loss: 0.7595, Validation Loss: 2.2258
Validation Loss Increased. Model Not Saved.
**************************************************


#### BLEU Score

In [59]:
from nltk.translate.bleu_score import SmoothingFunction

# Calculate BLEU score with smoothing
def calculate_bleu(predictions, references):
    smoothie = SmoothingFunction().method4
    bleu_scores = []
    for pred, ref in zip(predictions, references):
        bleu_scores.append(sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothie))
    return sum(bleu_scores) / len(bleu_scores)

In [60]:
# Test the model
bleu_score = test(model, test_loader, tokenizer, device)
print(f"\nBLEU Score: {bleu_score:.4f}")

Testing: 100%|██████████| 135/135 [00:24<00:00,  5.42it/s]



BLEU Score: 0.0648


In [61]:
# Create a simple QA bot
def qa_bot(context, question):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)
    answer = tokenizer.decode(input_ids[0][start_index:end_index+1])
    return answer

In [62]:
test_data[7]['story']

'CHAPTER XX \n\n\n\nHEDA\'S TALE \n\n\n\n\n\nThat evening when I was lying on my bed outside the cave, I heard the tale of Anscombe and Heda. Up to a certain point he told it, then she went on with the story. \n\n"On the morning after our arrival at this place, Allan," said Anscombe, "I woke up to find you gone from the hut. As you did not come back I concluded that you were with Zikali, and walked about looking for you. Then food was brought to us and Heda and I breakfasted together, after which we went to where we heard the horses neighing and found that yours was gone. Returning, much frightened, we met Nombe, who gave me your note which explained everything, and we inquired of her why this had been done and what was to become of us. She smiled and answered that we had better ask the first question of the king and the second of her master Zikali, and in the meanwhile be at peace since we were quite safe. \n\n"I tried to see Zikali but could not. Then I went to inspan the horses with

In [63]:
# Example usage of the QA bot
context = test_data[10]['story']
question = "When is the start of the Julian year?"
answer = qa_bot(context, question)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: When is the start of the Julian year?
Answer: 1 january


## Question Answering by Fine-Tuning GPT2

#### Imports

In [1]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
import nltk

nltk.download('punkt')

import logging
logging.disable(logging.WARNING)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#### Load Dataset and Process Dataset

In [2]:
def load_coqa_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data['data']

class CoQADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        context = item['story']
        question = item['questions'][0]['input_text']
        answer = item['answers'][0]['input_text']

        # Adjusted prompt format
        input_text = f"Context: {context} Question: {question} Answer briefly:"
        target_text = answer

        # Tokenize input and target
        inputs = self.tokenizer.encode_plus(
            input_text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        targets = self.tokenizer.encode_plus(
            target_text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': targets['input_ids'].flatten(),
        }


In [3]:
data = load_coqa_data('/content/drive/MyDrive/coqa-train-v1.0.json')
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

In [4]:
# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS token

train_dataset = CoQADataset(train_data, tokenizer)
val_dataset = CoQADataset(val_data, tokenizer)
test_dataset = CoQADataset(test_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size=8)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# Initialize model
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#### Training, Validation and Testing Function

In [6]:
def train(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc="Training")

    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({'loss': loss.item()})

    return total_loss / len(train_loader)

In [7]:
def validate(model, val_loader, device):
    model.eval()
    total_loss = 0
    progress_bar = tqdm(val_loader, desc="Validating")
    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            progress_bar.set_postfix({'loss': loss.item()})

    return total_loss / len(val_loader)

In [8]:
def test(model, test_loader, tokenizer, device):
    model.eval()
    all_predictions = []
    all_answers = []
    progress_bar = tqdm(test_loader, desc="Testing")
    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Generate concise answer
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=30,
                num_beams=3,
                temperature=0.5,
                early_stopping=True
            )
            prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

            all_predictions.append(prediction)

            # Decode true answer for BLEU score calculation
            true_answer = tokenizer.decode(batch['labels'][0], skip_special_tokens=True)
            all_answers.append(true_answer)

    bleu_score = calculate_bleu(all_predictions, all_answers)
    return bleu_score

#### BLEU Score

In [9]:
def calculate_bleu(predictions, references):
    bleu_scores = [sentence_bleu([ref.split()], pred.split()) for pred, ref in zip(predictions, references)]
    return sum(bleu_scores) / len(bleu_scores)

#### Training

In [10]:
from torch.cuda.amp import autocast, GradScaler
# Initialize GradScaler for mixed precision training
scaler = GradScaler()

<ipython-input-10-fb8d49ee7630>:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [13]:
# Training and evaluation function with model saving
def train_model(train_loader, val_loader, epochs=3):
    model.train()
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        total_train_loss = 0
        for batch in tqdm(train_loader):
            optimizer.zero_grad()

            # Move data to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass with mixed precision
            with autocast():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                total_train_loss += loss.item()

            # Backward pass and optimization with scaling
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        # Average train loss over the epoch
        avg_train_loss = total_train_loss / len(train_loader)
        print(f"Training loss: {avg_train_loss:.4f}")

        # Validation phase
        model.eval()
        total_val_loss = 0
        bleu_scores = []

        with torch.no_grad():
            for batch in tqdm(val_loader):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                # Forward pass
                with autocast():
                    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                    total_val_loss += loss.item()

                # Generate predictions for BLEU score evaluation
                generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=50)
                for i in range(len(generated_ids)):
                    predicted_text = tokenizer.decode(generated_ids[i], skip_special_tokens=True)
                    target_text = tokenizer.decode(labels[i], skip_special_tokens=True)
                    bleu_scores.append(sentence_bleu([target_text.split()], predicted_text.split()))

        # Average validation loss and BLEU score
        avg_val_loss = total_val_loss / len(val_loader)
        avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
        print(f"Validation loss: {avg_val_loss:.4f} - BLEU Score: {avg_bleu_score:.4f}")

train_model(train_loader, val_loader)

Epoch 1/3


  0%|          | 0/630 [00:00<?, ?it/s]<ipython-input-13-8fc903d02275>:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 630/630 [04:36<00:00,  2.28it/s]


Training loss: 0.0501


  0%|          | 0/135 [00:00<?, ?it/s]<ipython-input-13-8fc903d02275>:42: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0

Validation loss: 0.0517 - BLEU Score: 0.0020
Epoch 2/3


...

100%|██████████| 630/630 [04:26<00:00,  2.37it/s]


Training loss: 0.0467


100%|██████████| 135/135 [01:18<00:00,  1.72it/s]

Validation loss: 0.0529 - BLEU Score: 0.0020


RuntimeError: Parent directory /content/drive/MyDrive/model_save does not exist.

In [14]:
save_path = '/content/drive/MyDrive/gpt2_model.pth'

torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")

Model saved to /content/drive/MyDrive/gpt2_model.pth


#### Testing

In [15]:
# Test the model
bleu_score = test(model, test_loader, tokenizer, device)
print(f"BLEU Score: {bleu_score:.4f}")

Testing:   0%|          | 0/135 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Testing: 100%|██████████| 135/135 [02:28<00:00,  1.10s/it]


BLEU Score: 0.0031


In [16]:
# Create a simple QA bot
def qa_bot(context, question):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=20,  # Increase the token limit slightly
            num_beams=20,        # Increase the number of beams for better quality answers
            temperature=1,    # Adjust temperature for more focused responses
            early_stopping=True
        )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [17]:
test_data[3]['story']

'(CNN) -- Tiger Woods has split from the management company which has helped him become one of the world\'s wealthiest sportsmen since he joined it at the start of his professional golf career back in 1996. \n\nThe former top-ranked player announced on Monday that he will be staying with his longtime agent Mark Steinberg, who has left IMG after his contract expired. \n\n"Staying with Mark Steinberg. Total confidence in him," Woods wrote on social networking website Twitter. \n\n"Excited about the next stage in my professional life. Fond memories of Mark McCormack." \n\nMcCormack founded IMG, one of the world\'s leading sports management companies, but died in 2003. \n\nSliding Tiger drops out of golf\'s top ten \n\nWoods\' endorsements have been handled by Steinberg since 1998, but the golfer\'s declining fortunes have reportedly led to his agent\'s departure from IMG. \n\nWoods has not won a tournament since the end of 2009, and the scandal about his marital infidelities that broke af

In [20]:
# Example usage of the QA bot
context = test_data[3]['story']
question = "Who is Tiger Woods?"
answer = qa_bot(context, question)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: Who is Tiger Woods?
Answer: Who is Tiger Woods?(CNN) -- Tiger Woods has split from the management company which has helped him become one of the world's wealthiest sportsmen since he joined it at the start of his professional golf career back in 1996. 

The former top-ranked player announced on Monday that he will be staying with his longtime agent Mark Steinberg, who has left IMG after his contract expired. 

"Staying with Mark Steinberg. Total confidence in him," Woods wrote on social networking website Twitter. 

"Excited about the next stage in my professional life. Fond memories of Mark McCormack." 

McCormack founded IMG, one of the world's leading sports management companies, but died in 2003. 

Sliding Tiger drops out of golf's top ten 

Woods' endorsements have been handled by Steinberg since 1998, but the golfer's declining fortunes have reportedly led to his agent's departure from IMG. 

Woods has not won a tournament since the end of 2009, and the scandal about hi

## Gradio Interface

#### Imports

In [1]:
!pip install gradio

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, DistilBertTokenizer, DistilBertForQuestionAnswering, GPT2Tokenizer, GPT2LMHeadModel
import torch

#### Loading the Trained Models

In [4]:
# Load T5 from Google Drive
from google.colab import drive
drive.mount('/content/drive')
t5_model_path = '/content/drive/MyDrive/T5_finetuned_CoQA'
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_path)
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_path)

# Load DistilBERT from .pth file
distilbert_model_path = '/content/drive/MyDrive/distilbert_qa_model.pth'
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')
distilbert_model.load_state_dict(torch.load(distilbert_model_path, map_location=torch.device('cpu')))

# Load GPT-2 from .pth file
gpt2_model_path = '/content/drive/MyDrive/gpt2_model.pth'
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')
gpt2_model.load_state_dict(torch.load(gpt2_model_path, map_location=torch.device('cpu')))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-4-3989ffe85b0f>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. 

<All keys matched successfully>

#### QA Functions for all Models

In [5]:
def qa_t5(question, context):
    input_text = f"question: {question} context: {context}"
    input_ids = t5_tokenizer.encode(input_text, return_tensors="pt")
    outputs = t5_model.generate(input_ids)
    answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

def qa_distilbert(question, context):
    inputs = distilbert_tokenizer.encode_plus(question, context, return_tensors="pt")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    outputs = distilbert_model(input_ids=input_ids, attention_mask=attention_mask)
    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits)
    answer = distilbert_tokenizer.decode(input_ids[0][start_index:end_index+1])
    return answer

def qa_gpt2(question, context):
    input_text = f"{question} {context}"
    input_ids = gpt2_tokenizer.encode(input_text, return_tensors="pt")
    outputs = gpt2_model.generate(input_ids, max_length=200)
    answer = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

#### Gradio Interface

In [6]:
import gradio as gr

def qa_bot(model_name, context, question):
    if model_name == "T5":
        return qa_t5(question, context)
    elif model_name == "DistilBERT":
        return qa_distilbert(question, context)
    elif model_name == "GPT-2":
        return qa_gpt2(question, context)

interface = gr.Interface(
    fn=qa_bot,
    inputs=[
        gr.Radio(choices=["T5", "DistilBERT", "GPT-2"], label="Model"),  # Changed to gr.Radio
        gr.Textbox(lines=10, placeholder="Enter the article (context) here...", label="Context"),  # Changed to gr.Textbox
        gr.Textbox(lines=2, placeholder="Enter the question here...", label="Question")  # Changed to gr.Textbox
    ],
    outputs=gr.Textbox(label="Answer"),  # Changed to gr.Textbox
    title="Question Answering with Multiple Models",
    description="Select a model, enter the context, and ask a question to get the answer."
)

interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cb61a1b0d402b768ee.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---------------

## PRINTING

In [7]:
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  dvisvgm fonts-droid-fallback fonts-lato fonts-lmodern fonts-noto-mono fonts-texgyre
  fonts-urw-base35 libapache-pom-java libcmark-gfm-extensions0.29.0.gfm.3 libcmark-gfm0.29.0.gfm.3
  libcommons-logging-java libcommons-parent-java libfontbox-java libfontenc1 libgs9 libgs9-common
  libidn12 libijs-0.35 libjbig2dec0 libkpathsea6 libpdfbox-java libptexenc1 libruby3.0 libsynctex2
  libteckit0 libtexlua53 libtexluajit2 libwoff1 libzzip-0-13 lmodern pandoc-data poppler-data
  preview-latex-style rake ruby ruby-net-telnet ruby-rubygems ruby-webrick ruby-xmlrpc ruby3.0
  rubygems-integration t1utils teckit tex-common tex-gyre texlive-base texlive-binaries
  texlive-fonts-recommended texlive-latex-base texlive-latex-recommended texlive-pictures
  texlive-plain-generic tipa xfonts-encodings xfonts-utils
Suggested packages:
  fonts-noto fonts-fre

In [8]:
!jupyter nbconvert --to PDF "/content/drive/My Drive/Colab Notebooks/22070126061_NLP_Assignment_5.ipynb"

[NbConvertApp] Converting notebook /content/drive/My Drive/Colab Notebooks/22070126061_NLP_Assignment_5.ipynb to PDF
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('metadata' was unexpected)

Failed validating 'additionalProperties' in stream:

On instance['cells'][15]['outputs'][0]:
{'metadata': {'tags': None},
 'name': 'stderr',
 'output_type': 'stream',
 'text': 'Epoch 1:   0%|          | 0/900 [00:00<?, '
         '?it/s]/usr/local/lib/p...'}
[NbConvertApp] Writing 172945 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 141532 bytes to /content/drive/My Drive/Colab Notebooks/22070126061_NLP_Assignment_5.pdf
